In [20]:
from bot2 import *;
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate
from datetime import date
import plotly.express as px

In [21]:
def sql_fetch():
    con = sqlite3.connect(r'./hashtags.sqlite3')
    cursorObj = con.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    return [name[0] for name in cursorObj.fetchall() if '__' in name[0]]

In [22]:
sql_fetch()

['hashtag__12_03_23',
 'hashtag__14_03_23',
 'hashtag__18_03_23',
 'hashtag__19_03_23',
 'hashtag__26_03_23',
 'hashtag__11_04_23',
 'hashtag__14_04_23',
 'hashtag__09_05_23',
 'hashtag__26_05_23',
 'hashtag__27_05_23',
 'hashtag__28_05_23',
 'hashtag__30_05_23']

In [23]:
all_insta_tables = []
for name in sql_fetch():
        all_insta_tables.append(sqlToDf(name))
idf = pd.concat(all_insta_tables)

In [24]:
date_w_post = idf.groupby('date')['posts'].sum().reset_index()

In [25]:
X = date_w_post[['date']].copy()
y = date_w_post['posts']
X['ord'] = X.date.apply(lambda d: date.toordinal(d))
X.drop(columns=['date'], inplace=True)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
reg = GradientBoostingRegressor()
reg2 = LinearRegression()
cv1 = cross_validate(reg, X, y, cv=3, n_jobs=-1)
cv2 = cross_validate(reg2, X, y, cv=3, n_jobs=-1)
print(cv1['test_score'])
print(cv2['test_score'])

[-0.36042104 -0.26092025 -4.22007243]
[-0.0628045  -0.06388197 -0.33115556]


In [26]:
px.bar(y=[cv1['test_score'], cv2['test_score']])

{'fit_time': array([0.0320003 , 0.02299619, 0.0320003 ]),
 'score_time': array([0.00200129, 0.00300169, 0.00200129]),
 'test_score': array([-0.36042104, -0.26092025, -4.22007243])}